<a href="https://colab.research.google.com/github/uzusio/whisper_tool.ipynb/blob/main/whisper_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声文字起こしツール for whisper


---


## 概要
OpenAIの[whisper](https://github.com/openai/whisper)を使用した音声文字起こしツールです。Google Colabの無料環境で動作し、簡単な初期設定だけで高精度な文字起こしが可能です。


---


## 使い方

1.   編集->ノートブックの設定->ハードウェア アクセラレータ から「T4 GPU」を選択してください

2.   初期設定を入力してください
     - **INPUT_MEDIA_PATH**:  
       - `ファイルから読み込み`: 対象ファイルのパスを記入（例: `/content/hogehoge.mp3`）。  
       - `URLから読み込み`: 動画や音声のURLを記入（例: `http://example.com/video`）。  
     - **MODEL_TYPE**:  
       文字起こしの精度を指定します。高精度な`large-v3`を推奨。  
     - **srt_output**:  
       字幕ファイル（.srt形式）の生成を有効化したい場合にチェックを入れてください。

3. **セルを実行**
   - 初期設定を入力後、以降のセルを上から順に実行してください。または「ランタイム」→「以降のセルを実行」でまとめて実行可能です。

## リリースノート

### 2024.11.19
- UIを修正
- 一部ライブラリのバージョンを固定
- URLから動画をダウンロードできるように修正

### 2024.3.5
- Whisperを**faster-whisper**に切り替え、高速化。
- Papago翻訳APIの仕様変更に伴い一時的に選択肢から削除。
- マイク入力機能を削除。

### 2023.1.13
- マイクからの入力機能を追加。
- Google翻訳およびPapago翻訳を追加。
- Web上の動画をダウンロードする機能を追加。



In [ ]:
#@title ## Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

In [25]:
import os 

#@title ## 初期設定
#@markdown ## 基本設定:
#@markdown - メディアを入力（ファイルパスまたは動画ページのURLを入力）

#@markdown (Tips)左サイドバーにファイルをドラッグ＆ドロップ → 右クリックで『パスをコピー』
INPUT_MEDIA_PATH = "例: /path/to/media.mp4 もしくは http://example.com/movieid" #@param {type:"string"}
DIR_PATH = None
FILE_NAME = None

# 自動判定するINPUT_TYPE
if INPUT_MEDIA_PATH.startswith("http://") or INPUT_MEDIA_PATH.startswith("https://"):
    INPUT_TYPE = "URLから読み込み"
else:
    INPUT_TYPE = "ファイルから読み込み"
    DIR_PATH, FILE_NAME = os.path.split(INPUT_MEDIA_PATH)
    # ファイルの存在チェック
    if not os.path.exists(INPUT_MEDIA_PATH):
        raise FileNotFoundError(f"指定されたファイルが見つかりません: {INPUT_MEDIA_PATH}")

#@markdown - 利用するモデルサイズを選択
MODEL_TYPE="large-v3" #@param ["tiny", "base","small","medium","large","large-v2","large-v3"] {type:"string"}

#@markdown - 字幕ファイル(.srt)を出力したい場合下にチェック
srt_output=True #@param {type: "boolean"}

#@markdown ---


#@markdown ## 詳細設定:

#@markdown - 認識言語（メディアの言語）を指定（"auto"だと自動判定されます）
TRANS_FROM  = 'ja' #@param  ["auto","ja","en","ko","zh-CN","zh-TW","es","fr","vi","th","id"]

#@markdown - 翻訳機能を有効にする場合下にチェック
translate_option=False #@param {type: "boolean"}

#@markdown - 翻訳エンジンを指定
TRANSLATOR = 'GoogleTranslator' #@param ["GoogleTranslator", "PapagoTranslator"]

#@markdown - 翻訳先言語を指定
TRANS_TO  = 'ja' #@param  ["ja","en","ko","zh-CN","zh-TW","es","fr","vi","th","id"]

#@markdown - Webから動画ファイルも保存する
save_video=False #@param {type: "boolean"}


In [ ]:
#@title ## 必要ライブラリのインストール
# ctranslate2 のバージョンを固定してインストール
!pip install ctranslate2==4.4.0

!pip install faster_whisper==1.0.0
!pip install srt
!pip install deep-translator
!pip install yt-dlp

In [ ]:
#@title ## モデルのロード
#import whisper
from faster_whisper import WhisperModel

# model = whisper.load_model(MODEL_TYPE)
model = WhisperModel(MODEL_TYPE, device="cuda", compute_type="float16")

In [26]:
#@title ## 文字起こし実行
import subprocess

# 入力ファイルパス
file_path = INPUT_MEDIA_PATH

if INPUT_TYPE == "URLから読み込み":
    # ダウンロード形式オプション
    format_option = "mp4" if save_video else "mp3"
    yt_format = f"{'-f mp4' if save_video else '-x --audio-format mp3'}"

    # yt-dlpでファイル名を取得
    get_name_cmd = f"yt-dlp --get-filename -o '%(title)s.%(ext)s' {yt_format} {INPUT_MEDIA_PATH}"
    file_name = subprocess.check_output(get_name_cmd, shell=True).decode("utf-8").strip()

    # ダウンロード
    subprocess.run(f"yt-dlp {yt_format} -o '{file_name}' {INPUT_MEDIA_PATH}", shell=True)
    file_path = f"/content/{file_name}"

# Whisperで文字起こし
segments, info = model.transcribe(
    file_path,
    beam_size=5,
    vad_filter=True,
    without_timestamps=True,
    language=TRANS_FROM
)


In [29]:
import srt
from datetime import timedelta
from srt import Subtitle

def translate_segments(segments, translator, target_lang, source_lang='auto'):
    translated_segments = []

    for segment in segments:
        if translator == 'GoogleTranslator':
            translated_text = GoogleTranslator(source=source_lang, target=target_lang).translate(segment.text)
        # elif translator == 'PapagoTranslator':
        #     translated_text = Translator(CLIENT_ID, SECRET_KEY).translate(text=segment.text, target=target_lang, source=source_lang).text
        else:
            raise ValueError(f"Unknown translator: {translator}")

        translated_segment = {
            'text': translated_text,
            'start': segment.start,
            'end': segment.end
        }
        translated_segments.append(translated_segment)

    return translated_segments


def result2subs(segments, translate_option=False, translator=None, target_lang=None, source_lang='auto'):
    if translate_option:
        segments = translate_segments(segments, translator, target_lang, source_lang)
    else:
        # 翻訳オプションがOFFの場合、segmentsをそのままリストに変換
        segments = [{'start': segment.start, 'end': segment.end, 'text': segment.text} for segment in segments]

    subs = []

    for i, segment in enumerate(segments):
        start = segment['start']
        end = segment['end']
        text = segment['text']
        sub = Subtitle(index=i+1, start=timedelta(seconds=start), end=timedelta(seconds=end), content=text)
        subs.append(sub)

    return subs

# 字幕出力
if srt_output:
    if translate_option:
        # 翻訳された字幕ファイルのみを出力
        with open(f"{file_path}_{TRANS_TO}.srt", "w") as f:
            f.write(srt.compose(result2subs(segments, translate_option=True, translator=TRANSLATOR, target_lang=TRANS_TO, source_lang=TRANS_FROM)))
    else:
        # 元の言語の字幕ファイルを出力
        with open(f"{file_path}.srt", "w") as f:
            f.write(srt.compose(result2subs(segments)))
